In [ ]:
# TODO UPDATE THIS OLD MESSY CODE WITH NEW PANEL

In [3]:
# Imports
import pandas as pd
import numpy as np
import itertools
import time
import tensorflow as tf
import pickle
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping
from tensorflow.keras import initializers
from keras.models import Model
from sklearn.utils import resample
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [4]:
def subsetToAssetUniverseFull(df, asset_universe_dict, train_or_test):
    # determine the asset universe to use for whether train or test data
    if train_or_test == 'train':
        index_start = 0
        index_end   = len(asset_universe_dict)-4
    elif train_or_test=='test':
        index_start = len(asset_universe_dict)-4
        index_end   = len(asset_universe_dict)
    else:
        assert(False),('get wit zee program')
        
    # subset to included assets
    for i in range(index_start, index_end):
        # extract this quarter and its included assets
        date = list(asset_universe_dict.keys())[i]
        assets = asset_universe_dict[date]

        # form start and end date for this window
        start_date = datetime.strptime(date, '%Y-%m-%d')
        end_date   = datetime.strptime(date, '%Y-%m-%d') + relativedelta(months=3)

        # drop rows in this time period that are not the included assets
        df = df[~(((df.index>=start_date) & (df.index<end_date)) & (~df.asset.isin(assets)))]
    
    return df

In [5]:
def subsetToAssetUniversePerWeek(temp_df, asset_universe_dict, oos_week):
    # Determine what quarter the oos_week is in
    oos_mnth = pd.to_datetime(oos_week).month
    oos_yr   = pd.to_datetime(oos_week).year
    mnth_qtr = int(np.floor((oos_mnth-1)/3)*3+1)
    if mnth_qtr == 10:
        oos_qtr = str(oos_yr)+'-'+str(mnth_qtr)+'-01'
    else:
        oos_qtr = str(oos_yr)+'-0'+str(mnth_qtr)+'-01'

    # Determine the asset universe
    asset_universe = asset_universe_dict[oos_qtr]
    
    # Subset the training data to the asset universe
    temp_df = temp_df[temp_df.asset.isin(asset_universe)]
    
    return temp_df

In [6]:
def fitFFNN(train_df, hps_yhats_dict, val_df=None, early_stopping=True):
    # Obtain the covariates
    column_names = list(df.columns.values)
    column_names.remove('asset')
    column_names.remove('r_tplus7')
    covariates = column_names

    # Extract the hyperparameters
    number_hidden_layer = hps_yhats_dict['number_hidden_layer']
    learning_rate       = hps_yhats_dict['learning_rate']
    l1_penalty          = hps_yhats_dict['l1_penalty']
    batch_size          = hps_yhats_dict['batch_size']
    number_ensemble     = hps_yhats_dict['number_ensemble']
    boot_strap_pct      = hps_yhats_dict['boot_strap_pct']

    # Initialize the models
    models = []

    # Loop over the ensembles to build models for each
    assert(number_ensemble <= 5)
    n=round(boot_strap_pct*len(train_df))
    for i in range(0, number_ensemble):
        # Bootstrap the rows so different models in the ensemble are less correlated
        train_df = resample(train_df, replace=True, n_samples=n, random_state=i)
        
        # Obtain the training input and output data and, if passed, validation data
        train_rhs   = train_df[covariates]  
        train_y     = train_df[['r_tplus7']]
        if val_df is not None:
            val_rhs     = val_df[covariates]  
            val_y       = val_df[['r_tplus7']]
        
        # According to which model in the ensemble it is, initialize parameters.
        if i==0:
            weight_initializer=initializers.HeNormal(seed=i)
            bias_initializer=initializers.Constant(0.1)
        elif i==1:
            weight_initializer=initializers.GlorotUniform(seed=i)
            bias_initializer=initializers.Constant(0.1)
        elif i==2:
            weight_initializer=initializers.RandomUniform(seed=i)
            bias_initializer=initializers.Constant(0.1)
        elif i==3:
            weight_initializer=initializers.HeNormal(seed=i)
            bias_initializer=initializers.HeNormal(seed=i)
        elif i==4:
            weight_initializer=initializers.HeNormal(seed=i)
            bias_initializer=initializers.GlorotUniform(seed=i)
        else:
            weight_initializer=initializers.HeNormal(seed=i)
            bias_initializer=initializers.RandomUniform(seed=i)
        
        # Build the model
        model = tf.keras.models.Sequential()
        model.add(tf.keras.Input(shape=(train_rhs.shape[1],)))
        model.add(Dense(6, activation='relu',
                        kernel_regularizer=regularizers.l1(l1=l1_penalty),
                        kernel_initializer=weight_initializer,
                        bias_initializer=bias_initializer))
        model.add(BatchNormalization())
        if number_hidden_layer >= 2:
            model.add(Dense(5, activation='relu',
                            kernel_regularizer=regularizers.l1(l1=l1_penalty),
                            kernel_initializer=weight_initializer,
                            bias_initializer=bias_initializer))
            model.add(BatchNormalization())
        if number_hidden_layer == 3:
            model.add(Dense(4, activation='relu',
                            kernel_regularizer=regularizers.l1(l1=l1_penalty),
                            kernel_initializer=weight_initializer,
                            bias_initializer=bias_initializer))
            model.add(BatchNormalization())
        model.add(Dense(1, activation="linear"))

        # Compile the model
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='mean_squared_error',
                      metrics=['mse'])

        # Prepare early stopping object 
        es = EarlyStopping(monitor='val_mse', mode='min', verbose=2, patience=2)

        # Fit the model
        if early_stopping == True:
            model.fit(x=train_rhs, y=train_y, 
                      batch_size=batch_size,
                      validation_data=(val_rhs, val_y), 
                      epochs=5, verbose=1,
                      workers=24, callbacks=[es])
        else:
            model.fit(x=train_rhs, y=train_y, 
                      batch_size=batch_size,
                      epochs=5, verbose=1,
                      workers=24)

        models.append(model)

    return models

In [7]:
def genYhats(val_df, models):
    # Build RHS 
    column_names = list(val_df.columns.values)
    column_names.remove('asset')
    column_names.remove('r_tplus7')
    covariates   = column_names
    oos_rhs      = val_df[covariates]  
    
    #Loop over each model and fit to RHS for all rows
    yhats = []
    for model in models:
        yhats.append(model.predict(oos_rhs))
        
    # return the average of the predicted yhats
    return(np.mean(yhats, axis=0))

In [8]:
def runCV(df, asset_universe_dict, last_train_year=2017, val_end_year=2020):
    # Initialize hp result objects
    hps_yhats_dict_list = []
    hps_mse_df_list     = []
    
    # Initialize the hyperparameter grid  
    batch_sizes          = [64, 128, 256] # note: ensure powers of 2 for eff
    number_hidden_layers = [1, 3]
    learning_rates       = [1e-3, 1e-2, 1e-1] 
    l1_penalties         = [1, 0.1, 0.01]
    number_ensembles     = [5]
    boot_strap_pcts      = [0.99]   

    # Determine the weeks in the validation window  
    val_weeks = np.unique(df[(df.index.year > last_train_year)  
                             & (df.index.year <= val_end_year)].index.values)  

    # Generate yhats for every hyperparameter grid point
    for hps in itertools.product(batch_sizes,
                                 number_hidden_layers,
                                 learning_rates,
                                 l1_penalties,
                                 number_ensembles,
                                 boot_strap_pcts):
        hps_yhats_dict = {'batch_size': hps[0],
                          'number_hidden_layer': hps[1],
                          'learning_rate': hps[2],
                          'l1_penalty': hps[3],
                          'number_ensemble': hps[4],
                          'boot_strap_pct':hps[5],
                          'yhats': np.array([]),
                          'ys': np.array([])}
        print(hps_yhats_dict, '\n')
        tic = time.perf_counter()
        for val_week in val_weeks:
            print(val_week, '\n')
            train_df = df[df.index < val_week].copy()
            train_df = subsetToAssetUniversePerWeek(train_df, asset_universe_dict, oos_week=val_week)
            val_df   = df[df.index == val_week].copy()
            val_df   = subsetToAssetUniversePerWeek(val_df, asset_universe_dict, oos_week=val_week)
            
            models = fitFFNN(train_df, hps_yhats_dict, val_df=val_df, early_stopping=True)
            yhats  = genYhats(val_df, models)
            ys     = val_df.r_tplus7.values

            hps_yhats_dict['yhats'] = np.append(hps_yhats_dict['yhats'], yhats)
            hps_yhats_dict['ys']    = np.append(hps_yhats_dict['ys'], ys)

            val_ys_todate = hps_yhats_dict['ys'] 
            rw_val_mse    = np.mean(np.square(val_ys_todate))
            model_val_mse = np.mean(np.square(val_ys_todate-hps_yhats_dict['yhats']))
            print('\n val random walk mse: ' + str(rw_val_mse))
            print('\n val model mse: ' + str(model_val_mse))
            print('\n val model mse winning?: ' + str(model_val_mse < rw_val_mse))
            print('\n\n')
        # Save run time and space out result print out
        toc = time.perf_counter()
        print('\n\n\n')

        # Update lists of results for each HP point
        hps_yhats_dict_list.append(hps_yhats_dict)
        
        # Set up the CV results to save
        cv_results_dict = hps_yhats_dict.copy()
        del cv_results_dict['yhats']
        del cv_results_dict['ys']
        cv_results_dict['runtime_mins'] = round((toc - tic)/60, 0)
        cv_results_dict['mse'] = model_val_mse
        hps_mse_df_list.append(pd.DataFrame(cv_results_dict, index=[0]))

        # Save to a CSV
        cv_df = pd.concat(hps_mse_df_list, ignore_index=True)
        timestr = time.strftime("%Y%m%d_%H%M%S")
        fp = '../4-output/cv-results-ffn-' + timestr + '.csv'
        cv_df.to_csv(fp, index=False)

    return hps_yhats_dict_list

In [9]:
def labelPortfolioWeights(df):
    # assign tertiles
    np.random.seed(42)
    df['rand']    = np.random.uniform(size=df.shape[0])
    df = df.sort_values(by=['date', 'yhat'])
    df['ranking'] = df.groupby(['date']).cumcount()
    df['counts']  = 1
    df['total_assets_per_week'] = df.groupby('date').counts.transform('sum')
    df['ranking']               = df.ranking/df.total_assets_per_week
    df.loc[df.ranking < 1/3, 'prtfl_wght'] = 0
    df.loc[(df.ranking>=1/3) & 
           (df.ranking<2/3), 'prtfl_wght'] = 1/6
    df.loc[df.ranking>=2/3,  'prtfl_wght'] = 5/6
    df['prtfl_wght'] = 3*df.prtfl_wght/df.total_assets_per_week
    
    # clean up
    df = df.drop(['rand', 'ranking', 'counts',
                  'total_assets_per_week'], axis=1)
    
    # confirm portfolio weights roughly sum to 1 for each week
    assert(len(np.unique(df.index)) == 
           np.sum(np.isclose(df.groupby(['date']).prtfl_wght.sum(), 1,
                             rtol=1e-2, atol=1e-2)))

    return df


In [10]:
def calcAnnualTransactionCosts(df):
    # merge on the previous week's holdings to the new holdings
    temp_df = df.copy()
    temp_df = temp_df[temp_df.week_idx<np.max(temp_df.week_idx)]
    temp_df['week_idx'] = temp_df.week_idx+1
    temp_df = temp_df[['week_idx', 'asset', 'prtfl_wght']]
    temp_df = temp_df.rename(columns={'prtfl_wght': 'prtfl_wght_tm7'})
    df = df.merge(temp_df,
                  on=['week_idx', 'asset'],
                  how='outer',
                  validate='one_to_one')

    # calc weekly turnover and ensure it has the appropriate range
    df['asset_to'] = np.abs(df.prtfl_wght - df.prtfl_wght_tm7)
    to_df = df.groupby('week_idx')[['asset_to']].sum().reset_index()
    assert((np.min(to_df.asset_to)>=0) & (np.max(to_df.asset_to<=2)))

    # correct the first and last week valid for buying the initial port and liquidating
    to_df.loc[to_df.week_idx==106, 'asset_to'] = 1
    to_df = pd.concat((to_df, pd.DataFrame(data={'week_idx': [262],
                                                 'asset_to': 1})))
    to_df = to_df.reset_index(drop=True)

    # add transaction costs assuming maker and taker fee of 20 bps each
    to_df['tc'] = to_df.asset_to*0.002

    # return annualize transaction cost
    return -np.sum(to_df.tc)

In [11]:
def calcPortfolioReturn(r_df):
    num_wks  = r_df.shape[0]
    if np.sum(r_df.r.values <= -1)>=1:
        return -1
    else:
        tot_ret  = np.product(r_df.r.values+1)-1
        wkly_ret = (tot_ret+1)**(1/num_wks)-1
        annl_ret = (wkly_ret+1)**(52.18)-1
        return annl_ret

In [12]:
def calcPortfolioSharpe(df):
    wkly_sharpe = np.mean(df.r.values)/np.std(df.r.values)
    annl_sharpe = wkly_sharpe*np.sqrt(52.18)
    return annl_sharpe

In [13]:
def max_draw_down(df):
    cumulative_ret=(df.r+1).cumprod()
    roll_max=cumulative_ret.rolling(len(cumulative_ret), min_periods=1).max()
    daily_drawdown=cumulative_ret/roll_max
    max_daily_drawdown=daily_drawdown.min() - 1
    return max_daily_drawdown

In [14]:
def max_1_month_loss(df):
    max_loss=(df['r']+1).rolling(4).apply(np.prod)
    max_loss_minus=max_loss.min()-1
    return max_loss_minus

In [15]:
def GenTestYhats(df, opt_hps, test_year=2021): 
    test_weeks = np.unique(df[df.index.year == test_year].index.values)
    
    test_df = pd.DataFrame()

    for test_week in test_weeks:
        print(test_week, '\n')
        temp_df  = df[df.index <= test_week].copy()
        temp_df  = subsetToAssetUniversePerWeek(temp_df, asset_universe_dict, oos_week=test_week)
        train_df = temp_df[temp_df.index < test_week].copy()
        oos_df   = temp_df[temp_df.index == test_week].copy()
           
        models = fitFFNN(train_df, opt_hps, val_df=None, early_stopping=False)
        yhats  = genYhats(oos_df, models)
        
        oos_df = oos_df[['asset', 'r_tplus7']]
        oos_df['yhat'] = yhats
        test_df = pd.concat((test_df, oos_df))
        rw_mse = np.mean(np.square(test_df.r_tplus7.values))
        model_mse = np.mean(np.square(test_df.r_tplus7.values - test_df.yhat.values))
        print('\n test random walk mse: ' + str(rw_mse))
        print('\n test model mse: ' + str(model_mse))
        print('winning?: ' + str(model_mse < rw_mse))
        print('\n')
    
    return test_df


In [25]:
# import and clean up data
with open('../3-data/clean/asset_universe_dates_and_lists.pkl', 'rb') as handle:
    asset_universe_dict = pickle.load(handle)
input_fp = '../3-data/clean/panel_train.pkl'
df = pd.read_pickle(input_fp)
df = df.set_index('date')
#df = df.drop('week_idx', axis=1)
df = df.sort_values(by=['date', 'asset'])


In [ ]:
# run CV
hps_yhats_list = runCV(df, asset_universe_dict, last_train_year=2017, val_end_year=2020)

In [ ]:
# form validation period results
df = df[['asset', 'week_idx', 'r_tplus7']]
df = subsetToAssetUniverseFull(df, asset_universe_dict, train_or_test='train')
df = df[df.index.year >= 2018]
df['yhat'] = hps_yhats_list[0]['yhats']
df      = labelPortfolioWeights(df)
annl_tc = calcAnnualTransactionCosts(df)
df['r'] = df.prtfl_wght*df.r_tplus7
r_df    = df.groupby(['date'])[['r']].sum()
annl_ret = calcPortfolioReturn(r_df)
annl_sharpe = calcPortfolioSharpe(r_df)
max_dd = max_draw_down(r_df)
max_1mo_loss = max_1_month_loss(r_df)
print('annual transaction costs in simple return terms: ' + str(np.round(annl_tc, 4)))
print('annual simple return before trans costs: ' + str(np.round(annl_ret, 4)))
print('annual sharpe: '+str(np.round(annl_sharpe, 2)))
print('max drawdown : '+str(np.round(max_dd, 2)))
print('max one month loss : '+str(np.round(max_1mo_loss, 2)))

# TODO SCOPE OLD NOTEBOOK FOR LATEST NUMBERS

In [ ]:
# import and clean up data for test fitting
input_fp = '../3-data/clean/panel_oos.pkl'
df = pd.read_pickle(input_fp)
df = df.set_index('date')
df = df.sort_values(by=['date', 'asset'])
opt_hps = hps_yhats_list[0].copy()
del opt_hps['yhats']
del opt_hps['ys']
test_df = GenTestYhats(df, opt_hps) 
test_df = test_df.merge(df[['asset', 'week_idx']],
                        on=['date', 'asset'],
                        how='inner', 
                        validate='one_to_one')

# form test period results
test_df = labelPortfolioWeights(test_df)
annl_tc = calcAnnualTransactionCosts(test_df)
test_df['r'] = test_df.prtfl_wght*test_df.r_tplus7
r_df    = test_df.groupby(['date'])[['r']].sum()
annl_ret = calcPortfolioReturn(r_df)
annl_sharpe = calcPortfolioSharpe(r_df)
max_dd = max_draw_down(r_df)
max_1mo_loss = max_1_month_loss(r_df)
print('annual transaction costs in simple return terms: ' + str(np.round(annl_tc, 4)))
print('annual simple return before trans costs: ' + str(np.round(annl_ret, 4)))
print('annual sharpe: '+str(np.round(annl_sharpe, 2)))
print('max drawdown : '+str(np.round(max_dd, 2)))
print('max one month loss : '+str(np.round(max_1mo_loss, 2)))

# TODO SCOPE OLD NOTEBOOK FOR LATEST NUMBERS